In [1]:
import re
import jieba
import json
import pandas as pd
import numpy as np
import functions_v2
from collections import Counter, defaultdict
from gensim.models import word2vec

Building prefix dict from /Users/jlin/.pyenv/versions/kanjian/lib/python3.5/site-packages/jieba/dict.txt.big ...
Dumping model to file cache /var/folders/35/zb7fx0vn48z5c3t8v8kmfv9r0000gn/T/jieba.uf957e97c453f115ab85d0b40b2015182.cache
Loading model cost 3.203 seconds.
Prefix dict has been built succesfully.


In [18]:
source_path = '/Users/jlin/Lingtelli/kanjian/'
data_path = source_path + 'data/'
output_path = source_path + 'temp_cluster/'
dataset_file = 'new_cluster_result02.csv'
keyword_file = 'key_word_data.csv'

In [55]:
def extractInfo(c):
    searchObj = re.search( r'[a-z0-9]{8}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{12}(.*)$', c['comment_no_punc'], re.M|re.I)
    if searchObj is None:
        print('NO PATTERN MATCHED IN COMMENT %s' % c['comment_no_punc'])
        c['article_id'] = 'NO_DATA'
        c['tags'] = 'NO_DATA'
        c['comment_pure'] = c['comment_no_punc']
    else:
        c['article_id'] = c['comment_no_punc'][searchObj.start(): searchObj.end()].split(' ', 1)[0]
        c['tags'] = c['comment_no_punc'][searchObj.start(): searchObj.end()].split(' ', 1)[1]
        c['comment_pure'] = c['comment_no_punc'][: searchObj.start()]                                # 沒有標點符號的版本段落
    
    search_content = re.search( r'[a-z0-9]{8}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{12}(.*)$', c['comment'], re.M|re.I)
    if search_content is not None:
        c['content'] = c['comment'][: search_content.start()]
    else:
        c['content'] = c['comment']
    return c

In [19]:
# erase [nan], NaN in df_comment_tag['keywords']
def cleanKeywords(keywords):
    if (type(keywords) == float) or (len(keywords) > 0 and type(keywords[0]) is float):
        return []
    keywords = [x for x in keywords if str(x) != 'nan']
    return list(set(keywords))

In [20]:
def makeTitle(data):
    try:
        if np.isnan(data['WEB_TITLE']):
            return data['PAPER_TITLE']
    except:
        return data['WEB_TITLE']

In [21]:
def toDateForm(data):
    date_str = data['PUBLISH_DATE']# if data['PUBLISH_DATE'] != 'NO_VALUE' else data['CREATE_TIME']
    date = date_str.split(' ')[0]
    date_split = date.split('/')
    try:
        if len(date_split[1]) == 1:
            date_split[1] = '0' + date_split[1]
        if len(date_split[2]) == 1:
            date_split[2] = '0' + date_split[2]
        return ''.join(date_split)
    except:
        print('cannot convert date format of', data['UUID'])
        return date_split[0]

In [22]:
df = pd.read_csv(data_path + dataset_file, names=['count', 'comments'])
df_keyword = pd.read_csv(data_path + keyword_file, sep='\t', encoding='utf8')

In [23]:
keywords = df_keyword['WORD'].unique()

In [24]:
for k in keywords:
    if type(k) is not float:
        jieba.add_word(k)

In [25]:
# 把同一文章ID的WORD集合起來
df_keyword = df_keyword.groupby('ALL_UNIT_UUID')['WORD'].apply(list).reset_index()

In [26]:
df = df[df['count'] >= 1].reset_index()
df['comments'] = df['comments'].apply(functions_v2.strToSeries)

In [38]:
comments = list(set([c for comments in df['comments'].tolist() for c in comments]))
df_comment = pd.DataFrame({'comment': comments})
df_comment['comment_no_punc'] = df_comment['comment'].apply(functions_v2.removePunctuation)

In [56]:
df_comment = df_comment.apply(extractInfo, axis=1) 

NO PATTERN MATCHED IN COMMENT     functionw, d, s, m {        var f  d.getElementsByTagName'script'0
NO PATTERN MATCHED IN COMMENT           j  d.createElement'script'


In [57]:
# df_comment_tag 內包含各文章所屬的文章id, tags: claude那裡分析出的關鍵字, WORD: 該文章給定的關鍵字, comment
df_comment_tag = df_comment.merge(df_keyword, left_on='article_id', right_on='ALL_UNIT_UUID', how='left')
df_comment_tag = df_comment_tag[['content', 'comment_pure', 'article_id', 'WORD']]
df_comment_tag.rename(columns={'comment_pure': 'comment_no_punc', 'WORD': 'keywords'}, inplace=True)

In [58]:
df_comment_tag['keywords'] = df_comment_tag['keywords'].apply(cleanKeywords)

In [59]:
df_comment_tag['keywords_str'] = df_comment_tag['keywords'].apply(lambda x: ' '.join(x))

In [60]:
df_comment_tag['seg_comment'] = df_comment_tag['comment_no_punc'].apply(lambda x: jieba.lcut(x))

Get Article Information (Author, Publish Date, Title, Type, Subtype)

In [61]:
df_article_info = pd.read_csv('~/Downloads/kangjiantype/20160715-ch-article_data.csv')
df_article_info = df_article_info[pd.notnull(df_article_info['PUBLISH_DATE'])]

In [62]:
df_article_info['TITLE'] = df_article_info.apply(makeTitle, axis=1)

In [63]:
df_article_info['PUBLISH_DATE'] = df_article_info.apply(toDateForm, axis=1)

In [64]:
df_article_subtype = pd.read_csv('/Users/jlin/Downloads/kangjiantype/0727-article_type_ref.csv')        # article_data_uuid, sub_type_id
df_subtype_ref = pd.read_csv('/Users/jlin/Downloads/kangjiantype/0727-sub_type_code_utf8_clean.csv')    # 文章的子分類, 該子分類所屬的main_type
df_type_ref = pd.read_csv('/Users/jlin/Downloads/kangjiantype/0727-type_code.csv', encoding='big5')     # 文章的分類

In [65]:
# 抓到文章的子分類id
df_article_info_subtype = df_article_info.merge(
    df_article_subtype[['ARTICLE_DATA_UUID', 'SUB_TYPE']], left_on='UUID', right_on='ARTICLE_DATA_UUID', left_index=True, how='left')
print('所整理的資料共有', len(df_article_info_subtype['SUB_TYPE'].unique()), '種子項目')
print('全部共有', len(df_subtype_ref['IUUID'].unique()), '個子項目')
print('沒有資料不在全部子項目裡？', set(df_subtype_ref['IUUID'].unique()) > set(df_article_info_subtype['SUB_TYPE']), 
      ', 整理的資料多了', list(set(df_article_info_subtype['SUB_TYPE']) - set(df_subtype_ref['IUUID'].unique())))

所整理的資料共有 58 種子項目
全部共有 429 個子項目
沒有資料不在全部子項目裡？ False , 整理的資料多了 [nan]


In [66]:
# 抓到子分類的命名以及分類的id
df_article_info_subtype_name = df_article_info_subtype.merge(
    df_subtype_ref[['IUUID', 'NAME', 'MAIN_TYPE']], left_on='SUB_TYPE', right_on='IUUID', how='left')

In [67]:
# 抓到分類的命名
df_article_info_all_type = df_article_info_subtype_name.merge(
    df_type_ref[['UUID', 'NAME']], left_on='MAIN_TYPE', right_on='UUID', how='left')

In [68]:
# UUID_y: 文章分類項目的id, SUB_TYPE: 文章分類子項目的id
df_article_info_all_type = df_article_info_all_type[['UUID_x', 'TITLE', 'AUTHOR', 'PUBLISH_DATE', 'URL_ID', 'NAME_x', 'NAME_y']]
df_article_info_all_type.rename(
    columns={'UUID_x': 'article_id', 'TITLE': 'title', 'AUTHOR':'author', 'PUBLISH_DATE':'publish_date', 'URL_ID':'url', 'NAME_x':'category', 'NAME_y':'type'},
    inplace=True)
# subtype is category

In [69]:
print(len(df_comment_tag), len(df_comment_tag['article_id'].unique()))
print(len(df_article_info_all_type), len(df_article_info_all_type['article_id'].unique()))
df_dataset = df_comment_tag.merge(df_article_info_all_type, on='article_id', how='left')
print(len(df_dataset))

25736 7491
11733 11173
27914


In [70]:
df_dataset = df_dataset[
    ['content', 'article_id', 'keywords', 'keywords_str', 'title', 'author', 'publish_date', 'url', 'category']
].to_msgpack('df_dataset.msg')

In [3]:
df_dataset = pd.read_msgpack('df_dataset.msg')
df_dataset['keywords_str'] = df_dataset['keywords'].apply(lambda x: ' '.join(x))
df_dataset['comment_no_punc'] = df_dataset['content'].apply(functions_v2.removePunctuation)

In [163]:
df_dataset['publish_date'].apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:])

0        2016-07-01
1        2012-03-01
2        2016-05-24
3        2010-01-01
4        2012-05-01
5        2011-01-01
6        2011-11-01
7        2011-11-01
8        2015-03-01
9        2015-08-10
10       2000-11-01
11       2015-03-04
12       2011-05-27
13       2015-11-05
14       2015-10-30
15       2013-09-01
16       1998-11-01
17       2002-04-01
18       2016-04-27
19       2013-02-01
20       2009-06-01
21       2016-02-01
22       2009-06-01
23       2011-07-13
24       2011-07-13
25       2014-08-01
26       2014-11-01
27       2007-04-01
28       2012-01-01
29       2016-01-13
            ...    
27884    2008-01-01
27885    2008-06-01
27886    2014-09-04
27887    2010-09-01
27888    2016-03-01
27889    2007-02-01
27890    2009-08-01
27891    2009-08-01
27892    2015-02-02
27893    2013-01-01
27894    2015-03-01
27895    2010-04-01
27896    2014-08-26
27897    2012-10-01
27898    2014-05-20
27899    2001-12-01
27900    2006-05-01
27901    2005-07-01
27902    2009-04-01


train word2vec model

In [103]:
sentences = df_comment_tag['seg_comment'].tolist()

In [104]:
w2v_model = word2vec.Word2Vec(sentences,)
w2v_model.init_sims(replace=True)

In [105]:
w2v_model.save('w2v_model.mdl')

In [4]:
w2v_model = word2vec.Word2Vec.load('w2v_model.mdl')

Query

In [5]:
# 很快的查詢方法，目前僅支援一個input
#query_list的詞有出現在內文或關鍵字
def keywordQuery(query_list, return_size=100):
    query = '|'.join(query_list)
    tdf = df_dataset[(df_dataset.comment.str.contains(query, na=False)) | (df_dataset.keywords.str.contains(query, na=False))]
    keyword_contain_query = [k for kws in tdf.keywords.str for k in kws if (type(k) is not float) and (k not in query)]
    return tdf, Counter(keyword_contain_query).most_common(return_size)

In [6]:
# 利用word2vec查出與_query_list內所有query相近的詞與為一群
def getConceptWords(words, threshold=0.65):
    concepts = defaultdict(list)
    for w in words:
        #print('\n\n', w, '->', end=' ')
        try:
            most_similar = w2v_model.most_similar(w, topn=9)
            concepts[w] = [m[0] for m in most_similar if m[1] >= threshold] + [w]
        except:
            continue
        #print(concepts[w])
    return concepts

In [7]:
# 該文章有這個keyword，但是該段落完全沒有類似的字眼，是否該回傳？ 還是回傳整段文章？
# 把文章合併或是照文章編排
def subkeywordQuery(query_list, concepts):
    query = '|'.join(query_list)
    tdf = df_dataset[(df_dataset.comment.str.contains(query, na=False)) | (df_dataset.keywords_str.str.contains(query, na=False))]
    
    for concept, words in concepts.items():
        print(concept, 'size:', end='')
        _query = '|'.join(words)
        temp_df = tdf[tdf.keywords_str.str.contains(_query, na=False)]
        print(len(temp_df))
        print(temp_df)

In [8]:
def allFieldsQuery(query_list, keywords=None, author=None, publish_date=None, categories=None, most_common_size=10, cluster_idx=0, batch_size=5):
    query = '|'.join(query_list)
    tdf = df_dataset[(df_dataset.comment_no_punc.str.contains(query, na=False)) | (df_dataset.keywords_str.str.contains(query, na=False))]
    
    keyword_contain_query = [k for kws in tdf.keywords.str for k in kws if (type(k) is not float) and (k not in query)]
    top_keywords = [w[0] for w in Counter(keyword_contain_query).most_common(most_common_size)]

    if author is not None:
        tdf = tdf[(tdf.author.str.contains(author, na=False))]
    
    if categories is not None:
        query = '|'.join(categories)
        tdf = tdf[(tdf.category.str.contains(query, na=False))]
    
    if publish_date is not None:
        start = ''.join(publish_date[0].split('-'))
        end = ''.join(publish_date[1].split('-'))
        mask = (tdf['publish_date'] >= start) & (tdf['publish_date'] <= end)
        tdf = tdf.loc[mask]
    
    clusters = []
    concept_list = []
    if keywords is not None:
        concepts = getConceptWords(keywords)
    else:
        concepts = getConceptWords(top_keywords[:most_common_size])
        
    for concept, words in concepts.items():
        _query = '|'.join(words)
        #print('\n\n', _query)
        print(concept, 'size:', end='')
        temp_df = tdf[tdf.keywords_str.str.contains(_query, na=False)]
        print(len(temp_df))
        if len(temp_df) > 0:
            clusters.append(temp_df)
            concept_list.append(concept)
            #print(temp_df)
    return clusters[cluster_idx: cluster_idx + batch_size], concept_list[cluster_idx: cluster_idx + batch_size]


In [9]:
def convertJSON(df_list, keywords):
    clusters = []
    
    for df, keyword in zip(df_list, keywords):
        cluster_obj = dict()
        cluster_obj['keyword'] = keyword
        cluster_obj['size'] = len(df)
        temp_df = df[['content', 'article_id', 'title', 'author', 'publish_date', 'url', 'category', 'keywords']]
        cluster_obj['member'] = temp_df.to_dict(orient='records')
        clusters.append(cluster_obj)
        
    return clusters

In [13]:
cc, c = allFieldsQuery(['水災'])

義診 size:1
安養機構 size:2
醫療 size:1
基改食品 size:1
台灣 size:1
火災 size:2
高樓層 size:2
消防 size:2


In [14]:
clusters_json = convertJSON(cc, c)

In [15]:
clusters_json

[{'keyword': '義診',
  'member': [{'article_id': '065ece36-6a2e-4059-8a96-d09e90c9bc63',
    'author': '林芝安',
    'category': '醫療迷思',
    'content': '清晨6點不到，所有團員在駐紮一晚的蒙古包外面集合，壓緊帽子、裹著禦寒大衣，逐一清點好相關急救醫療物品後，開始往看來不算艱困的山上挺進。 沒多久，才發現這座山會騙人，因為坡度不小，中途得休息好幾回，除了體力過人的領隊唐高駿一馬當先之外，大夥無不氣喘吁吁，餓著肚子繼續攻堅。在陣陣冷風中，總算攀上山頭，準備進行災難醫療救援訓練（Disaster Medical Assistant Team , DMAT）。 大夥各自就定位，有人扮演傷患、有人負責通報、有人忙著綁繩結、還有些人負責製作擔架……隨著分秒流逝，突然，朝陽從遙遠山頭探出臉來，頃刻間，第一道曙光射出，金色光芒遍灑山頭，然後將腳下的大地染成黃紅一片，在山上忙個不停的我們，有如沐浴在光中。 除了義診工作，接受災難醫療救援訓練也是本次義診團的主要任務之一，所有醫護人員都必須參與演習。 當災難發生時，如何在最短的時間內進行緊急救護與醫療照顧，將傷害減至最低，平時的訓練與演習就變得很重要，尤其台灣，地震、水災等各種災難頻繁，一旦發生重大災難，醫療救援隊必須緊急進駐，協助災區復原。 台北市立聯合醫院災難醫療救援隊利用這次外蒙義診，培訓醫護人員為種子志工，進行野外訓練。 就地取材，用枯木做擔架 台北市立聯合醫院忠孝院區院長兼總院區緊急醫療部部長唐高駿說，難得有機會移師外蒙古山區，可練習如何就地取材，利用當地既有的資源進行應變與救援。說著說著，唐高駿轉身往山坡樹林方向跑，沒多久，他肩上扛著長條白樺樹枯木回來，利用兩根木頭將三件救援背心串起來，就完成了克難擔架。 模擬事件：登山意外 醫護人員趕抵現場，檢傷後得知，一位重傷（疑似脊椎受傷）、兩位輕傷（一位手傷，一位腳傷）。 先在山上對重傷病患進行頸椎固定，使用繩結將患者綑綁在擔架上，初步評估有無心跳、血壓後，趕緊利用無線通報醫療站人員待命，之後用枯木做成的擔架將重傷患者搬運下山，另外兩位輕傷患者則用三角巾簡單固定傷口，由醫護人員陪同，緩步下山。 一抵達山下醫療站（設於蒙古包內），待命人員隨即對重

In [232]:
json.dump(clusters_json, open('test.json', 'w'))#, ensure_ascii=False)

In [ ]:
##########################################################################################################################################

In [301]:
#word_vectors = [w2v_model[key] for key in w2v_model.vocab if key in keywords]

In [343]:
def sumVectors(words):
    sentence_vector = []
    seg = jieba.lcut(comment)
    for w in words:
        if w in w2v_model.vocab:
            sentence_vector.append(w2v_model[w])
    return np.sum(sentence_vector, axis=0)

In [394]:
df_comment_tag['vector'] = df_comment_tag['seg_comment'].apply(sumVectors)

In [347]:
from sklearn.cluster import AgglomerativeClustering

In [351]:
agglo_model = AgglomerativeClustering(n_clusters=1500, affinity='cosine', linkage='complete')

In [397]:
predicted_labels = agglo_model.fit_predict(df_comment_tag['vector'].tolist())

ValueError: Cannot extract more clusters than samples: 1500 clusters where given for a tree with 1000 leaves.

In [408]:
y3 = w2v_model.most_similar(['治療'], topn=50)

In [409]:
print(y3)

[('藥物治療', 0.7594596743583679), ('治療方式', 0.7414567470550537), ('手術', 0.7131774425506592), ('手術治療', 0.6928289532661438), ('療法', 0.689993143081665), ('癌症治療', 0.6893002986907959), ('治療方法', 0.6822962164878845), ('化學治療', 0.6813525557518005), ('中醫治療', 0.6758089065551758), ('化療', 0.6729283332824707), ('用藥', 0.6710823774337769), ('復發', 0.66049724817276), ('診斷', 0.6587794423103333), ('早期診斷', 0.653386116027832), ('處置', 0.6483652591705322), ('病程', 0.642364501953125), ('診療', 0.6410486698150635), ('治癒', 0.637911319732666), ('標靶藥物', 0.6375629901885986), ('放射線治療', 0.6278493404388428), ('西醫', 0.6252554655075073), ('標靶治療', 0.6243441104888916), ('開刀', 0.6167346835136414), ('肝癌', 0.6061704754829407), ('復健', 0.6058024168014526), ('急性期', 0.6047356724739075), ('診治', 0.6030184030532837), ('晚期', 0.6015010476112366), ('術後', 0.6006958484649658), ('療程', 0.5985381603240967), ('根治', 0.5973528027534485), ('放射治療', 0.595122218132019), ('洗腎', 0.593602180480957), ('放血', 0.589836597442627), ('病情', 0.5884119272232056), ('